# Eigenvalues of the dielectric matrix

We compute a few eigenvalues of the dielectric matrix ($q=0$, $ω=0$) iteratively.

In [1]:
using DFTK
using Plots
using KrylovKit
using Printf

# Calculation parameters
kgrid = [1, 1, 1]
Ecut = 5

# Silicon lattice
a = 10.26
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

# Compute the dielectric operator without symmetries
model  = model_LDA(lattice, atoms, positions, symmetries=false)
basis  = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis, tol=1e-8);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.233502745491                   -0.50    7.0         
  2   -7.249451642861       -1.80       -1.38    1.0   7.97ms
  3   -7.250712019136       -2.90       -1.71    2.0   9.79ms
  4   -7.251018138647       -3.51       -1.89    1.0   7.92ms
  5   -7.251315275530       -3.53       -2.47    1.0   7.96ms
  6   -7.251337395059       -4.66       -3.07    1.0   7.90ms
  7   -7.251338671783       -5.89       -3.40    2.0   10.2ms
  8   -7.251338775028       -6.99       -3.82    1.0   8.18ms
  9   -7.251338793367       -7.74       -4.17    2.0   9.74ms
 10   -7.251338797967       -8.34       -4.69    1.0   8.48ms
 11   -7.251338798633       -9.18       -5.15    2.0   10.3ms
 12   -7.251338798701      -10.17       -5.75    2.0   9.93ms
 13   -7.251338798704      -11.48       -6.15    2.0   10.0ms
 14   -7.251338798704      -12.34       -6.67    2.0   9.73ms
 15   -7.

Applying $ε^† ≔ (1- χ_0 K)$ …

In [2]:
function eps_fun(δρ)
    δV = apply_kernel(basis, δρ; ρ=scfres.ρ)
    χ0δV = apply_χ0(scfres, δV)
    δρ - χ0δV
end;

… eagerly diagonalizes the subspace matrix at each iteration

In [3]:
eigsolve(eps_fun, randn(size(scfres.ρ)), 5, :LM; eager=true, verbosity=3);

[ Info: Arnoldi iteration step 1: normres = 0.053540097616857046
[ Info: Arnoldi iteration step 2: normres = 0.5928098747133606
[ Info: Arnoldi iteration step 3: normres = 0.865043596988595
[ Info: Arnoldi iteration step 4: normres = 0.2530994291736357
[ Info: Arnoldi iteration step 5: normres = 0.5621051794522524
[ Info: Arnoldi schursolve in iter 1, krylovdim = 5: 0 values converged, normres = (3.01e-02, 8.19e-02, 4.87e-01, 2.66e-01, 1.23e-02)
[ Info: Arnoldi iteration step 6: normres = 0.2376813732706695
[ Info: Arnoldi schursolve in iter 1, krylovdim = 6: 0 values converged, normres = (5.96e-03, 1.11e-01, 1.70e-01, 8.96e-02, 3.18e-02)
[ Info: Arnoldi iteration step 7: normres = 0.1091325015497849
[ Info: Arnoldi schursolve in iter 1, krylovdim = 7: 0 values converged, normres = (3.18e-04, 1.22e-02, 1.58e-02, 6.55e-02, 7.91e-02)
[ Info: Arnoldi iteration step 8: normres = 0.10683636723068957
[ Info: Arnoldi schursolve in iter 1, krylovdim = 8: 0 values converged, normres = (1.47e-05